# Configurar directorios

In [1]:
DATADIR = 'era5/' # directory containing downloaded era5 data
FIREDATADIR = 'fire_danger/' # directory containing fire data
DESTDIR = 'processed_data/' # directory to save .npy files for each time step and variable

# Cargar bibliotecas

In [2]:
import numpy as np
import netCDF4 as nc
import os
from tqdm.notebook import tqdm

# Variables de configuración

In [3]:
vars = ['u10','v10','t2m','lai_hv','lai_lv','tp','fdimrk'] #considered variables (see 0_download_data.ipynb for long names)
months = [(1,31),(2,28),(12,31)] # months + days in month in dowloaded era5 .nc files
years = np.arange(2002,2023) # downloaded years

# Procesamiento de datos para crear archivos .npy

In [ ]:
# Processing Data to create .npy files
for var in vars:
  if not os.path.exists(DESTDIR + f"{var}"):
    os.makedirs(DESTDIR + f"{var}")
  
  for year in tqdm(years):
    if var=='fdimrk':
      root = nc.Dataset(FIREDATADIR + f"{year:d}.nc", 'r')
    else:
      root = nc.Dataset(DATADIR + f"{year:d}.nc", 'r')
    v = root.variables[var][:,:-9,:-5] #crop to get to a size suitable for the considered Unet-like model, here 140x140
    v = v.data
    root.close()
    if var in ['tp']: #change unit from m to mm for precipitation 
      v = 1000 * v
    t = 0 # time step within v array that I am working on
    for month, days in months:
      for day in range(days):
        np.save(DESTDIR + f"{var}/{year}_{month:02d}_{day+1:02d}.npy",v[t])
        t += 1

# Guardar información de latitud/longitud

In [7]:
root = nc.Dataset(DATADIR + f"2002.nc", 'r') #constant in time -> take from any year
lat = root.variables['latitude'][:-9].data #crop to get to a size suitable for the considered Unet-like model
lon = root.variables['longitude'][:-5].data 
np.save(DESTDIR + 'lat.npy', lat)
np.save(DESTDIR + 'lon.npy', lon)